Here, we are going to do dipole fits - as this is quite heavy processing you can use **8 CPUs**  
We are doing it on the *somato* dataset, but you have all the ingredients to apply it to your own data

In [ ]:
#%% PACKAGES

import mne
from os.path import join
from os import chdir
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
%matplotlib widget


In [ ]:
#%% SET DEFAULT PLOTTING PARAMETERS

mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['font.size'] = 12
mpl.rcParams['font.weight'] = 'bold'
mpl.rcParams['lines.linewidth'] = 3
plt.ion()

This data was added to UCloud

In [ ]:
somato_path = '/work/MEG_data/MNE-somato-data'
somato_meg_path = join(somato_path, 'sub-01', 'meg')
subjects_dir = join(somato_path, 'derivatives', 'freesurfer', 'subjects')
subject = 'somato'
chdir(somato_meg_path)

# dataset with 111 somatosensory stimulations of the left hand
# https://mne.tools/stable/overview/datasets_index.html#somatosensory

Just get a quick and dirty evoked response

In [ ]:
#%% GET EVOKED RESPONSE

raw_somato = mne.io.read_raw(join(somato_meg_path,
                                  'sub-01_task-somato_meg.fif'))

events_somato = mne.find_events(raw_somato)

event_id = dict(somato=1) # a somatosensory stimulation
tmin = -0.200 # s
tmax =  0.600 # s
baseline = (None, -0.010)

epochs_somato = mne.Epochs(raw_somato, events_somato, event_id, tmin, tmax,
                           baseline)

evoked_somato = epochs_somato.average()
evoked_somato.plot(xlim=(-0.050, 0.150))

Define the bem model needed for calculation of the lead field

In [ ]:
#%% DIPOLE FITTING

## boundary element method

## describe the surfaces and their conductivies
bem_model = mne.bem.make_bem_model(subject='01', subjects_dir=subjects_dir,
                                   conductivity=[0.3]) ## single layer model
## plot the three surfaces
mne.viz.plot_bem(subject, subjects_dir)


Load the pre-defined solution, or create it yourself (you have to uncomment; took about 2½ minutes with 8 CPUs)

In [ ]:

## model how the currents spread throughout the brain
#bem_solution = mne.bem.make_bem_solution(bem_model)

bem_solution = mne.bem.read_bem_solution(join(subjects_dir, 'somato', 'bem', '01-5120-bem-sol.fif'))

## trans 
trans = join(somato_path, 'derivatives', 'sub-01', 'somato-trans.fif')


Whitening the data from the magnetometers and the gradiometers based on the baseline period
Note that the real rank of the data doesn't correspond to the nominal rank

In [ ]:

## noise_covariance - needed for whitening the channels, i.e. normalizing
# the output of the magnetometers and the gradiometers such that they are
# comparable

noise_cov_somato = mne.compute_covariance(epochs_somato, tmin=None, tmax=-0.010,
                                          rank='info')
noise_cov_somato.plot(epochs_somato.info)

Cropping the data - we don't wanna fit it all, i.e. we think dipoles are best for the early, sensory responses

In [ ]:
## dipole modelling

evoked_somato_cropped = evoked_somato.copy() ## create a copy to be cropped
evoked_somato_cropped.crop(0.000, 0.150) ## only look at this time interval

evoked_somato_cropped.plot()


The lead field and the source model are computed on the fly based on the bem solution.

In [ ]:

## dipole fit - a forward model (lead field) and a (volumetric) source model
## are computed on the fly 
 
dip, residual = mne.dipole.fit_dipole(evoked_somato_cropped, noise_cov_somato,
                                      bem=bem_solution, trans=trans,
                                      n_jobs=-1)

Plotting different metrics for evaluating the dipole fit

In [ ]:
dip.plot_locations(trans=trans, subject=subject, subjects_dir=subjects_dir,
                   show_all=False) # show the dipole with the best fit
fig = plt.figure()
plt.plot(dip.times, dip.gof)
plt.title('Goodness of fit of dipole models')
plt.xlabel('Time (s)'); plt.ylabel('Goodness of fit') # 100 is max 0 is min

fig = plt.figure()
plt.plot(dip.times, dip.amplitude * 1e9)
plt.title('Amplitude of fitted dipoles')
plt.xlabel('Time (s)'); plt.ylabel('Current density (nAm)')
plt.show()

Doing a sequential fit, i.e. fitting a new dipole model on the residuals of the first fit:  
  - first fit, minimise: $fit_0 = (\boldsymbol b(t) - \hat{\boldsymbol b}(t)_{fit_0})^2$
  - second fit, minimise: $fit_1 =[(\boldsymbol{b}(t)- \hat{\boldsymbol b}(t)_{fit_0}) - \hat{\boldsymbol b}(t)_{fit_1}]^2$
  - ... and so on, *ad nauseam*

In [ ]:
#%% sequential fit

dip_no2, residual_no2 = mne.dipole.fit_dipole(residual, noise_cov_somato,
                                      bem=bem_solution, trans=trans,
                                      n_jobs=-1)

modelled_response_no2 = residual.copy()
modelled_response_no2._data -= residual_no2.data

fig = plt.figure()
plt.plot(dip_no2.times, dip_no2.gof)
plt.title('Goodness of fit of dipole models')
plt.xlabel('Time (s)'); plt.ylabel('Goodness of fit') # 100 is max 0 is min

fig = plt.figure()
plt.plot(dip_no2.times, dip_no2.amplitude * 1e9)
plt.title('Amplitude of fitted dipoles')
plt.xlabel('Time (s)'); plt.ylabel('Current density (nAm)')
plt.show()

modelled_response_no2.plot()
dip_no2.plot_locations(trans=trans, subject=subject, subjects_dir=subjects_dir,
                   show_all=False) # show the dipole with the best fit

dip_no2_SII = dip_no2.copy()
dip_no2_SII.crop(0.090, 0.110)
dip_no2_SII.plot_locations(trans=trans, subject=subject, subjects_dir=subjects_dir,
                   show_all=False) # show the dipole with the best fit

Some experimental code inspired from: https://mne.tools/stable/auto_examples/inverse/multi_dipole_model.html#sphx-glr-auto-examples-inverse-multi-dipole-model-py, to extract time courses from epochs instead of evokeds, such that you could apply it in machine learning

In [ ]:
cropped_dip = dip.copy()
cropped_dip.crop(0.040, 0.040)


fwd, _ = mne.make_forward_dipole(dip, bem_solution, epochs_somato.info, trans=trans)


In [ ]:

# Apply MNE inverse
inv = mne.minimum_norm.make_inverse_operator(epochs_somato.info, fwd, noise_cov_somato, fixed=True, depth=0)


In [ ]:

cropped_epochs_somato = epochs_somato.copy()
cropped_epochs_somato.load_data()
cropped_epochs_somato.crop(0.000, 0.150)
stcs_40_ms = mne.minimum_norm.apply_inverse_epochs(cropped_epochs_somato, inv, lambda2=1, method="MNE")


In [ ]:

plt.figure()
avg_data = np.zeros(shape=(len(stcs_40_ms), len(cropped_epochs_somato.times)))
for stc_index, stc in enumerate(stcs_40_ms):
    this_data = np.mean(stc.data, axis=0) * 1e9
    plt.plot(stc.times * 1e3, this_data)
    avg_data[stc_index, :] = this_data
    
plt.plot(stc.times * 1e3, np.mean(avg_data, axis=0) , 'k--', lw=10 )
plt.title('Individual epochs and and their average')
plt.xlabel('Time (ms)')
plt.ylabel('Current Density (nAm)')
plt.show()
#avg_data.shape